In [47]:
import pandas as pd
from binance.client import Client
from binancekeys import api_key, secret_key
import time
from datetime import datetime
import plotly.graph_objects as go

In [48]:
client = Client(api_key, secret_key)

In [49]:
#coins = ['BTC', 'ETH', 'LTC']
coins = ['XMR']

merge = False
for coin in coins:
    cols = ['OpenTime',
        f'{coin}-USD_Open',
        f'{coin}-USD_High',
        f'{coin}-USD_Low',
        f'{coin}-USD_Close',
        f'{coin}-USD_Volume',
        'CloseTime',
        f'{coin}-QuoteAssetVolume',
        f'{coin}-NumberOfTrades',
        f'{coin}-TBBAV',
        f'{coin}-TBQAV',
        f'{coin}-ignore']
    print(f'gathering {coin}...')
    
    symbol = f'{coin}USDT'
    interval = client.KLINE_INTERVAL_1HOUR
    start_str = 'Jun 19, 2022'
    end_str = 'Jul 06, 2022'
    csv_filename = 'XMR_Jun1922_Jul0622_1h.csv'
    
    klines = client.get_historical_klines(symbol, interval, start_str, end_str)
    
    coin_df = pd.DataFrame(klines, columns=cols)
    
    if merge:
        all_coins_df = pd.merge(coin_df, all_coins_df, how='inner',on=['OpenTime','CloseTime'])
    else:
        all_coins_df = coin_df
        merge = True
        
    time.sleep(10)

gathering XMR...


In [50]:
all_coins_df['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in all_coins_df['OpenTime']]
all_coins_df['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in all_coins_df['CloseTime']]

In [51]:
for col in all_coins_df.columns:
    if not 'Time' in col:
        all_coins_df[col] = all_coins_df[col].astype(float)

In [52]:
all_coins_df

,OpenTime,XMR-USD_Open,XMR-USD_High,XMR-USD_Low,XMR-USD_Close,XMR-USD_Volume,CloseTime,XMR-QuoteAssetVolume,XMR-NumberOfTrades,XMR-TBBAV,XMR-TBQAV,XMR-ignore
0,2022-06-19 01:00:00,104.4,105.4,103.1,103.1,3374.825,2022-06-19 01:59:59.999,351356.5236,1319.0,1247.940,130094.7352,0.0
1,2022-06-19 02:00:00,103.2,103.6,101.2,102.0,2472.156,2022-06-19 02:59:59.999,252510.3378,1072.0,1097.165,112353.4334,0.0
2,2022-06-19 03:00:00,101.9,102.2,100.5,101.4,2426.675,2022-06-19 03:59:59.999,246203.3631,881.0,1405.403,142738.5224,0.0
3,2022-06-19 04:00:00,101.5,103.0,101.5,101.9,2244.345,2022-06-19 04:59:59.999,229742.7388,736.0,1355.811,138850.7928,0.0
4,2022-06-19 05:00:00,102.0,104.1,101.8,103.4,3310.668,2022-06-19 05:59:59.999,341441.0803,1344.0,1902.506,196162.6544,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
404,2022-07-05 21:00:00,123.6,124.6,123.2,123.3,3558.969,2022-07-05 21:59:59.999,441293.8962,1480.0,2392.189,296697.1128,0.0
405,2022-07-05 22:00:00,123.3,123.6,122.4,123.6,1155.538,2022-07-05 22:59:59.999,142127.4498,797.0,536.464,66018.1644,0.0
406,2022-07-05 23:00:00,123.6,123.8,122.6,123.7,2620.222,2022-07-05 23:59:59.999,322728.2073,1579.0,1143.321,140970.3965,0.0
407,2022-07-06 00:00:00,123.7,123.9,122.0,122.6,4264.594,2022-07-06 00:59:59.999,522429.6748,1763.0,2519.651,308659.9938,0.0


In [53]:
list(all_coins_df.columns)

['OpenTime',
 'XMR-USD_Open',
 'XMR-USD_High',
 'XMR-USD_Low',
 'XMR-USD_Close',
 'XMR-USD_Volume',
 'CloseTime',
 'XMR-QuoteAssetVolume',
 'XMR-NumberOfTrades',
 'XMR-TBBAV',
 'XMR-TBQAV',
 'XMR-ignore']

In [54]:
for coin in coins:
    print(f'Plot coin {coin}...')
    fig = go.Figure(data=[go.Candlestick(x=all_coins_df['OpenTime'],
                open=all_coins_df[f'{coin}-USD_Open'],
                high=all_coins_df[f'{coin}-USD_High'],
                low=all_coins_df[f'{coin}-USD_Low'],
                close=all_coins_df[f'{coin}-USD_Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()

Plot coin XMR...


In [55]:
all_coins_df.to_csv(csv_filename, index=False)